In [126]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/student-performance-data/student_data.csv


In [127]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
import statistics as stat

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [128]:
df = pd.read_csv('../input/student-performance-data/student_data.csv')
df = df.dropna()
df.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3'],
      dtype='object')

In [ ]:
y1 = df['G1']
y2 = df['G2']
y3 = df['G3']
ytot = y1 + y2 + y3
y = ytot / 58

In [ ]:
sns.scatterplot(x = df['absences'], y = y)

In [ ]:
X = df.drop(columns = ['G1', 'G2', 'G3'])
X.columns
X.head()
XH = X

In [ ]:
"""ALL CATS: ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']"""
cats = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']
#XNoCats = X.drop(columns = cats)
#XNoCats.head()

In [ ]:
#ONEHOTENC TEST
transformer = make_column_transformer(
    (OneHotEncoder(), cats),
    remainder='passthrough')

transformed = transformer.fit_transform(df)
transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
XH = transformed_df
XH.head()

In [204]:
%%time

N = tf.keras.layers.Normalization()
N.adapt(np.array(XH))
linear_model = tf.keras.Sequential([
    N,
    layers.Dense(units=2, activation = 'softmax')
])
linear_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002),
    loss='mean_absolute_error')

CPU times: user 255 ms, sys: 3.02 ms, total: 258 ms
Wall time: 256 ms


In [210]:
%%time
history = linear_model.fit(
    XH,
    y,
    epochs=20,
    # Suppress logging.
    verbose=1,
    # Calculate validation results on 10% of the training data.
    validation_split = 0.1)

Epoch 1/20
12/12 [==============================] - 0s 11ms/step - loss: 0.1601 - val_loss: 0.1750
Epoch 2/20
12/12 [==============================] - 0s 5ms/step - loss: 0.1601 - val_loss: 0.1752
Epoch 3/20
12/12 [==============================] - 0s 6ms/step - loss: 0.1602 - val_loss: 0.1754
Epoch 4/20
12/12 [==============================] - 0s 5ms/step - loss: 0.1601 - val_loss: 0.1752
Epoch 5/20
12/12 [==============================] - 0s 6ms/step - loss: 0.1601 - val_loss: 0.1753
Epoch 6/20
12/12 [==============================] - 0s 6ms/step - loss: 0.1601 - val_loss: 0.1752
Epoch 7/20
12/12 [==============================] - 0s 5ms/step - loss: 0.1601 - val_loss: 0.1750
Epoch 8/20
12/12 [==============================] - 0s 6ms/step - loss: 0.1601 - val_loss: 0.1751
Epoch 9/20
12/12 [==============================] - 0s 6ms/step - loss: 0.1600 - val_loss: 0.1749
Epoch 10/20
12/12 [==============================] - 0s 5ms/step - loss: 0.1601 - val_loss: 0.1747
Epoch 11/20
12/12 

In [207]:
linear_model.evaluate(
    XH, y, verbose=1)

13/13 [==============================] - 0s 1ms/step - loss: 0.1616


0.1615842878818512

In [182]:
linear_model.save('MDL4')
MDL4 = tf.keras.models.load_model('MDL4')

In [211]:
MDL4.evaluate(
    XH, y, batch_size=None, verbose=0)

0.16047978401184082

In [212]:
#If it doesn't work well --> high bias. Check last entries for high variance
predModel =  MDL4.predict(X)
print("FIRST 20")
for i in range(20):
    pred = round(predModel[i][0] * 58)
    yi = round(y[i] * 58)
    dif = round(abs(yi-pred))
    print(f"Actual: {yi}, Pred: {pred}, Diff: {dif}")
print("LAST 20")
for i in range(len(X)-20, len(X)):
    pred = round(predModel[i][0] * 58)
    yi = round(y[i] * 58)
    dif = round(abs(yi-pred))
    print(f"Actual: {yi}, Pred: {pred}, Diff: {dif}")

FIRST 20
Actual: 17, Pred: 28, Diff: 11
Actual: 16, Pred: 29, Diff: 13
Actual: 25, Pred: 31, Diff: 6
Actual: 44, Pred: 29, Diff: 15
Actual: 26, Pred: 28, Diff: 2
Actual: 45, Pred: 27, Diff: 18
Actual: 35, Pred: 29, Diff: 6
Actual: 17, Pred: 29, Diff: 12
Actual: 53, Pred: 28, Diff: 25
Actual: 44, Pred: 28, Diff: 16
Actual: 27, Pred: 30, Diff: 3
Actual: 34, Pred: 29, Diff: 5
Actual: 42, Pred: 28, Diff: 14
Actual: 31, Pred: 29, Diff: 2
Actual: 46, Pred: 27, Diff: 19
Actual: 42, Pred: 30, Diff: 12
Actual: 41, Pred: 29, Diff: 12
Actual: 28, Pred: 28, Diff: 0
Actual: 16, Pred: 29, Diff: 13
Actual: 28, Pred: 29, Diff: 1
LAST 20
Actual: 26, Pred: 27, Diff: 1
Actual: 44, Pred: 24, Diff: 20
Actual: 27, Pred: 26, Diff: 1
Actual: 45, Pred: 26, Diff: 19
Actual: 30, Pred: 25, Diff: 5
Actual: 43, Pred: 24, Diff: 19
Actual: 20, Pred: 25, Diff: 5
Actual: 32, Pred: 25, Diff: 7
Actual: 11, Pred: 26, Diff: 15
Actual: 16, Pred: 25, Diff: 9
Actual: 29, Pred: 26, Diff: 3
Actual: 17, Pred: 27, Diff: 10
Actual

In [213]:
predModel =  MDL4.predict(X)
diffList = []
for i in range(len(X)):
    pred = round(predModel[i][0] * 58)
    yi = round(y[i] * 58)
    diff = round(abs(yi-pred))
    diffList.append(diff)
#Mean
sum(diffList) / len(diffList)

9.883544303797468